## Steps 
- Import cost & energy library 
- calculate energy and Li content for standard cells (NMC622, NMC811, LFP, SSB) 
- take assumption for 2030 distribution
- calculate extrem in case only one chemistry exists 

In [1]:
import math
import sys
sys.path.insert(0, '/home/matthias/Documents/GitHub/Cell_model_energy_costs')
import cell_model as Cellmodel
import numpy as np
import pandas as pd

## Define materials 

In [2]:
# Active materials
NMC811_cathode = Cellmodel.Activematerial_cathode("NMC811" , 
                                                  195, # specificy capacity 
                                                  3.86 # average E vs Li
                                                 )
NMC622_cathode = Cellmodel.Activematerial_cathode("NMC622" , 181, 3.86)
LFP_cathode = Cellmodel.Activematerial_cathode("LFP", 160, 3.3)
Gr = Cellmodel.Activematerial_anode("Gr", 344 , 0.17)
LiM = Cellmodel.Activematerial_anode("LiMetall", 3862, 0.0)

## Current Collectors, Eleytrolytes and Separators

In [3]:
### Others
## Currentcollectors 
Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)


## Electrolyte 
LP40_standard = Cellmodel.Electrolyte_liquid("LiPF6" , 
                                             "EC:DMC", 
                                             1.1, # molar concentration mol/L 
                                             151.9 # molarmass salt 
                                            )

LLZO_elyte = Cellmodel.Electrolyte_solid("LLZO",
                                         838.5, # molarmass electrolyte
                                         6.94, # molarmass
                                         7, # index Li in LLZO
                                         0.20, # fraction of SE in pos electrode
                                         5.1 # density in g cm-3
                                        )

Elyte_liquid = Cellmodel.Electrolyte_liquid("","",0,0) # need to define as a empty container for the function ...
                                                        # to work 
# Separator
Separator_standard = Cellmodel.Separator("PP+Al" , 
                                         12e-4 , 
                                         0.44 , 
                                         1.18
                                        )
Separator_LLZO = Cellmodel.Separator("LLZO", 
                                     40e-4, # thickness
                                     0.7, # porosity
                                     5.1 # density g cm-3
                                    ) 

## Electrodes

In [4]:
# Electrods 
positive_NMC811 = Cellmodel.Electrodecomposition_cathode_opt1(NMC811_cathode.name , 
                                                              3.3 , # areal capacity 
                                                              0.95 , # active fraction 
                                                              3.4, # density electrode
                                                              NMC811_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5
positive_NMC811_ASSB = Cellmodel.Electrodecomposition_cathode_opt1(NMC811_cathode.name , 
                                                              3.3 , 
                                                              (positive_NMC811.active_frac - 
                                                               LLZO_elyte.pos_electrode_fraction), 
                                                              3.4, 
                                                              NMC811_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5

positive_NMC622 = Cellmodel.Electrodecomposition_cathode_opt1(NMC622_cathode.name , 
                                                              3.3 , 
                                                              0.95 , 
                                                              3.4, 
                                                              NMC622_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5

positive_NMC622_ASSB = Cellmodel.Electrodecomposition_cathode_opt1(NMC622_cathode.name , 
                                                              3.3 , 
                                                              (positive_NMC622.active_frac - 
                                                               LLZO_elyte.pos_electrode_fraction), 
                                                              3.4, 
                                                              NMC622_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5

positive_LFP = Cellmodel.Electrodecomposition_cathode_opt1(LFP_cathode.name , 
                                                           2.5 , 
                                                           0.95 ,
                                                           2.5, 
                                                           LFP_cathode
                                                          ) # NMC = 3.4 / LFP = 2.5

positive_LFP_ASSB = Cellmodel.Electrodecomposition_cathode_opt1(LFP_cathode.name , 
                                                              2.5 , 
                                                              (positive_LFP.active_frac - 
                                                               LLZO_elyte.pos_electrode_fraction), 
                                                              3.4, 
                                                              LFP_cathode
                                                             ) # NMC = 3.4 / LFP = 2.5

negative_Gr = Cellmodel.Electrodecomposition_anode_opt1(Gr.name , 
                                                        3.3*1.1 , 
                                                        0.965 , 
                                                        1.6, 
                                                        Gr
                                                       )

negative_LiM = Cellmodel.Electrodecomposition_anode_opt1(LiM.name , 
                                                         10.32 , 
                                                         1.00 , 
                                                         0.543, 
                                                         LiM
                                                        ) # 50µm == 10.32 mAh/cm2

## Combining all into cells

In [5]:
### Total cells
# Graphite Anode
NMC811Gr_cell = Cellmodel.Pouch("NMC811-Gr_P", 
                                positive_NMC811, 
                                negative_Gr, 
                                Separator_standard , 
                                LP40_standard, 
                                1.7, #  electrolyte capacity ratio  ml/Ah 
                                30, 
                                10, 
                                35 , # layers
                                113e-4, # pouch thickness
                                1.8 , # pouch density 
                                1.0, # pouchclearance
                                2 , # terminal height 
                                5 , # terminal width 
                                15e-4, # therminal thickness
                                2.76 , # terminal density pos  
                                8.96, # terminal density neg 
                                10, # extra mass 
                                0.93, # 
                                NMC811_cathode,
                                Gr
                                )

NMC622Gr_cell = Cellmodel.Pouch("NMC622-Gr_P", 
                                    positive_NMC622, 
                                    negative_Gr, 
                                    Separator_standard , 
                                    LP40_standard, 
                                    1.7, #  electrolyte capacity ratio  ml/Ah 
                                    30, 
                                    10, 
                                    35 , # layers
                                    113e-4, # pouch thickness
                                    1.8 , # pouch density 
                                    1.0, # pouchclearance
                                    2 , # terminal height 
                                    5 , # terminal width 
                                    15e-4, # therminal thickness
                                    2.76 , # terminal density pos  
                                    8.96, # terminal density neg 
                                    10, # extra mass 
                                    0.93, # 
                                    NMC622_cathode, # activmaterial cathode
                                    Gr # active material anode
                                     )

LFPGr_cell = Cellmodel.Pouch("LFP-Gr_P", 
                                    positive_LFP, 
                                    negative_Gr, 
                                    Separator_standard , 
                                    LP40_standard, 
                                    1.7, #  electrolyte capacity ratio  ml/Ah 
                                    30, 
                                    10, 
                                    35 , # layers
                                    113e-4, # pouch thickness
                                    1.8 , # pouch density 
                                    1.0, # pouchclearance
                                    2 , # terminal height 
                                    5 , # terminal width 
                                    15e-4, # therminal thickness
                                    2.76 , # terminal density pos  
                                    8.96, # terminal density neg 
                                    10, # extra mass 
                                    0.93, # 
                                    LFP_cathode,
                                    Gr
                                  )

# Li metall Anode
NMC811LiM_cell = Cellmodel.Pouch("NMC811-LiM_P", 
                                    positive_NMC811_ASSB, 
                                    negative_LiM, 
                                    Separator_LLZO , 
                                    LLZO_elyte, 
                                    0, #  electrolyte capacity ratio  ml/Ah 
                                    30, 
                                    10, 
                                    35 , # layers
                                    113e-4, # pouch thickness
                                    1.8 , # pouch density 
                                    1.0, # pouchclearance
                                    2 , # terminal height 
                                    5 , # terminal width 
                                    15e-4, # therminal thickness
                                    2.76 , # terminal density pos  
                                    8.96, # terminal density neg 
                                    10, # extra mass 
                                    0.93, # 
                                    NMC811_cathode,
                                    LiM
                                      )

NMC622LiM_cell = Cellmodel.Pouch("NMC622-LiM_P", 
                                        positive_NMC622_ASSB, 
                                        negative_LiM, 
                                        Separator_LLZO , 
                                        LLZO_elyte,
                                        0, #  electrolyte capacity ratio  ml/Ah 
                                        30, 
                                        10, 
                                        35 , # layers
                                        113e-4, # pouch thickness
                                        1.8 , # pouch density 
                                        1.0, # pouchclearance
                                        2 , # terminal height 
                                        5 , # terminal width 
                                        15e-4, # therminal thickness
                                        2.76 , # terminal density pos  
                                        8.96, # terminal density neg 
                                        10, # extra mass 
                                        0.93, # 
                                        NMC622_cathode,
                                        LiM
                                      )

LFPLiM_cell = Cellmodel.Pouch("LFP-LiM_P", 
                                        positive_LFP_ASSB, 
                                        negative_LiM, 
                                        Separator_LLZO , 
                                        LLZO_elyte, 
                                        0, #  electrolyte capacity ratio  ml/Ah 
                                        30, 
                                        10, 
                                        35 , # layers
                                        113e-4, # pouch thickness
                                        1.8 , # pouch density 
                                        1.0, # pouchclearance
                                        2 , # terminal height 
                                        5 , # terminal width 
                                        15e-4, # therminal thickness
                                        2.76 , # terminal density pos  
                                        8.96, # terminal density neg 
                                        10, # extra mass 
                                        0.93, # 
                                        LFP_cathode,
                                        LiM
                                   )

In [6]:
all_cells = [NMC811Gr_cell,
                NMC622Gr_cell, 
                LFPGr_cell,
                NMC811LiM_cell,
                NMC622LiM_cell,
                LFPLiM_cell]

In [7]:
all_cells_details = []
idx = 0
df_all = []

for idx in range(0,len(all_cells)):
    one = [all_cells[idx].name_pouch, 
           all_cells[idx].energy, 
           all_cells[idx].capacity,
           all_cells[idx].positive.thickness_el_c,
           all_cells[idx].negative.thickness_el_a
          ]
    all_cells_details.append(one)
    # print(one)
    
df_all = pd.DataFrame(data = all_cells_details)
df_all.set_axis(['Name', 'Energy', 'Capacity', 'ThicknessCathode', 'ThicknessAnode'], axis=1, inplace = True)
df_all

,Name,Energy,Capacity,ThicknessCathode,ThicknessAnode
0,NMC811-Gr_P,237.81681,64.449,0.005239,0.006834
1,NMC622-Gr_P,237.81681,64.449,0.005645,0.006834
2,LFP-Gr_P,152.82225,48.825,0.006579,0.006834
3,NMC811-LiM_P,248.77314,64.449,0.006637,0.004921
4,NMC622-LiM_P,248.77314,64.449,0.007150,0.004921
5,LFP-LiM_P,161.12250,48.825,0.006127,0.004921




## Get the Masses

In [8]:
positives_liq = [positive_NMC811, positive_NMC622, positive_LFP]

positives_assb = [positive_NMC811_ASSB, positive_NMC622_ASSB, positive_LFP_ASSB]

negatives = [negative_Gr, negative_LiM]

cathodes = [NMC811_cathode, NMC622_cathode, LFP_cathode]

cells = [NMC811Gr_cell, NMC622Gr_cell, LFPGr_cell, NMC811LiM_cell, NMC622LiM_cell, LFPLiM_cell]

In [9]:
## Masses
cellmass_NMC811Gr, pouch_mass_NMC811Gr = Cellmodel.getMass_Pouch_total(positive_NMC811, negative_Gr, Al, Cu, Separator_standard, NMC811Gr_cell, LP40_standard)
cellmass_NMC622Gr, pouch_mass_NMC622Gr = Cellmodel.getMass_Pouch_total(positive_NMC622, negative_Gr, Al, Cu, Separator_standard, NMC622Gr_cell, LP40_standard)
cellmass_LFPGr, pouch_mass_LFPGr = Cellmodel.getMass_Pouch_total(positive_LFP, negative_Gr, Al, Cu, Separator_standard, LFPGr_cell, LP40_standard)
#
cellmass_NMC811LiM, pouch_mass_NMC811LiM = Cellmodel.getMass_Pouch_total(positive_NMC811_ASSB, 
                                                  negative_LiM, 
                                                  Al, 
                                                  Cu, 
                                                  Separator_LLZO, 
                                                  NMC811LiM_cell, 
                                                  LLZO_elyte)

cellmass_NMC622LiM, pouch_mass_NMC622Gr = Cellmodel.getMass_Pouch_total(positive_NMC622_ASSB, 
                                                                        negative_LiM, 
                                                                        Al, 
                                                                        Cu, 
                                                                        Separator_LLZO, 
                                                                        NMC622LiM_cell, 
                                                                        LLZO_elyte)
cellmass_LFPLiM, pouch_mass_LFPGr = Cellmodel.getMass_Pouch_total(positive_LFP_ASSB, 
                                                                  negative_LiM, 
                                                                  Al, 
                                                                  Cu, 
                                                                  Separator_LLZO, 
                                                                  LFPLiM_cell, 
                                                                  LLZO_elyte)


## Cathode Masses
# Cylindrical
cathode_mass_NMC811 = Cellmodel.getMass_cathode(positive_NMC811) * NMC811Gr_cell.jr_area * 2
cathode_mass_NMC622 = Cellmodel.getMass_cathode(positive_NMC622) * NMC622Gr_cell.jr_area * 2
cathode_mass_LFP = Cellmodel.getMass_cathode(positive_LFP) * LFPGr_cell.jr_area * 2
#
cathode_mass_NMC811_ASSB = Cellmodel.getMass_cathode(positive_NMC811_ASSB) * NMC811LiM_cell.jr_area * 2
cathode_mass_NMC622_ASSB = Cellmodel.getMass_cathode(positive_NMC622_ASSB) * NMC622LiM_cell.jr_area * 2
cathode_mass_LFP_ASSB = Cellmodel.getMass_cathode(positive_LFP_ASSB) * LFPLiM_cell.jr_area * 2


## Seperator and Elyte
Separator_mass = Cellmodel.getMass_separator(Separator_standard)

# ASSB
Separator_mass_LLZO = Cellmodel.getMass_separator(Separator_LLZO)
Li_abs_elyte_NMC811 = Cellmodel.getMass_elements_elyte_solid(LLZO_elyte, positive_NMC811_ASSB)
Li_abs_elyte_NMC622 = Cellmodel.getMass_elements_elyte_solid(LLZO_elyte, positive_NMC622_ASSB)
Li_abs_elyte_LFP = Cellmodel.getMass_elements_elyte_solid(LLZO_elyte, positive_LFP_ASSB)


pos_elec_materials = np.zeros((3,5))
neg_elec_materials = np.zeros((3,2))
Elyte_Li_mass = np.zeros((3,1))


In [20]:
Li_abs_elyte_NMC811

0.05098437686344663

In [18]:
Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)

### Materials
## Cylindrical
i = 0
for positive in positives_liq:
    Elyte_mass = Cellmodel.getMass_electrolyte_liquid(LP40_standard, cells[i])
    Li_Elyte_mass = Cellmodel.getMass_elements_pouch_elyte(LP40_standard, Separator_standard, cells[i])
    Ni, Co, Mn, Li, Al = Cellmodel.getMass_elements_pouch_c(positives_liq[i], Al, cathodes[i], cells[i])
    Gr, Cu, Binder_a = Cellmodel.getMass_elements_pouch_a(negative_Gr, Cu, cells[i])
    pos_elec_materials[i,] = [Ni, Co, Mn, Li, Al]
    neg_elec_materials[i,] = [Gr, Cu]
    Elyte_Li_mass[i,] = [Li_Elyte_mass]
    i = i+1
    Al = Cellmodel.CurrentCollector_cathode("Al", 14e-4 , 2.76)
    Cu = Cellmodel.CurrentCollector_anode("Cu", 8e-4 , 8.96)
# Cathode materials
df_pos_elec_materials = pd.DataFrame(data = pos_elec_materials, 
                                     index = [cells[0].name_pouch, cells[1].name_pouch, cells[2].name_pouch])
df_pos_elec_materials.set_axis(['Ni', 'Co', 'Mn', 'Li', 'Al'], axis=1, inplace = True)
# Anode materials
df_neg_elec_materials = pd.DataFrame(data = neg_elec_materials, 
                                     index = [cells[0].name_pouch, cells[1].name_pouch, cells[2].name_pouch])
df_neg_elec_materials.set_axis(['Gr', 'Cu'], axis=1, inplace = True)
# Elyte materials
df_Elyte_l_mass = pd.DataFrame(data = Elyte_Li_mass, 
                             index = [cells[0].name_pouch, cells[1].name_pouch, cells[2].name_pouch])
df_Elyte_l_mass.set_axis(['Li_elyte_mass'], axis=1, inplace = True)

g / mol * mol / L * L = g

Molare Masse * concentration * Elektrolytvolume = Mass Salz

Masse Electrolyte [g] * Massenanteil Lisalz [%g]

In [19]:
df_Elyte_l_mass

,Li_elyte_mass
NMC811-Gr_P,0.836627
NMC622-Gr_P,0.836627
LFP-Gr_P,0.633808
